In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os

# ========================================
# 1. Set Up Configuration and Parameters
# ========================================

# Ensure reproducibility
manualSeed = 999
tf.random.set_seed(manualSeed)
np.random.seed(manualSeed)

# Generator parameters (must match the trained model)
nz = 100  # Size of the latent vector (input to generator)
image_size = 64  # Size of the generated images
nc = 3    # Number of channels in the generated images (CIFAR-10 is RGB)

# Output directory
os.makedirs('latent_interpolation_results', exist_ok=True)

# ========================================
# 2. Load the Pre-trained Generator
# ========================================

# Load pre-trained generator model
if os.path.exists('generator.h5'):
    generator = tf.keras.models.load_model('generator.h5')
    generator.summary()
    print("Pre-trained generator model loaded.")
else:
    print("Error: Pre-trained generator model 'generator.h5' not found.")
    print("Please train the DCGAN model first and save the generator's model as 'generator.h5'.")
    exit()

# ========================================
# 3. Select Two Latent Vectors
# ========================================

# Generate two random latent vectors
z_dim = nz  # Should match the generator's input size
z1 = tf.random.normal([1, z_dim], seed=manualSeed)
z2 = tf.random.normal([1, z_dim], seed=manualSeed + 1)  # Different seed for a different vector

# ========================================
# 4. Interpolate Between the Vectors
# ========================================

# Number of interpolation steps
num_interpolations = 10

# Linearly interpolate between z1 and z2
interpolated_z = []
alphas = np.linspace(0, 1, num_interpolations)
for alpha in alphas:
    z = (1 - alpha) * z1 + alpha * z2
    interpolated_z.append(z)

# Convert list to Tensor
interpolated_z = tf.concat(interpolated_z, axis=0)

# ========================================
# 5. Generate Images from Interpolated Vectors
# ========================================

# Generate images
generated_images = generator(interpolated_z, training=False)
# Rescale images from [-1, 1] to [0, 1]
generated_images = (generated_images + 1) / 2.0

# ========================================
# 6. Visualize the Resulting Images
# ========================================

# Function to create a grid of images
def create_image_grid(images, num_cols):
    num_images = images.shape[0]
    num_rows = num_images // num_cols
    height, width = images.shape[1], images.shape[2]
    grid = np.zeros((height * num_rows, width * num_cols, images.shape[3]))
    for idx, img in enumerate(images):
        row = idx // num_cols
        col = idx % num_cols
        grid[row * height:(row + 1) * height, col * width:(col + 1) * width, :] = img.numpy()
    return grid

# Create the grid image
grid_image = create_image_grid(generated_images, num_cols=num_interpolations)

# Save the grid image
plt.figure(figsize=(20, 5))
plt.axis('off')
plt.title('Latent Space Interpolation')
plt.imshow(grid_image)
plt.savefig('latent_interpolation_results/interpolation.png', bbox_inches='tight')
plt.show()

print("Interpolation image saved to 'latent_interpolation_results/interpolation.png'.")

# ========================================
# 7. Comment on the Transitions
# ========================================

print("The interpolation shows a smooth transition between two generated images.")
print("Observe how the features in the images gradually change from one to the other.")
